# Set connection with GitHub

In [1]:
# ! cd /kaggle/working && rm -rf RECsys_Challenge2024

In [2]:
from kaggle_secrets import UserSecretsClient

token = UserSecretsClient().get_secret("Token")

! git clone https://{token}@github.com/madratak/RECsys_Challenge2024.git

Cloning into 'RECsys_Challenge2024'...
remote: Enumerating objects: 6039, done.
remote: Counting objects: 100% (332/332), done.
remote: Compressing objects: 100% (292/292), done.
remote: Total 6039 (delta 132), reused 20 (delta 7), pack-reused 5707 (from 3)
Receiving objects: 100% (6039/6039), 395.79 MiB | 22.68 MiB/s, done.
Resolving deltas: 100% (3355/3355), done.
Updating files: 100% (485/485), done.


In [3]:
! pip install PyGithub requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.9/375.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 28.4 MB/s eta 0:00:00


In [4]:
%cd /kaggle/working/RECsys_Challenge2024 
! python run_compile_all_cython.py

/kaggle/working/RECsys_Challenge2024
run_compile_all_cython: Found 11 Cython files in 5 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/usr/bin/python3'
Compiling [1/11]: MatrixFactorizationImpressions_Cython_Epoch.pyx... 
In file included from /usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/ndarraytypes.h:1929,
                 from /usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from MatrixFactorizationImpressions_Cython_Epoch.c:1252:
/usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wcpp-Wcpp]8;;]
   17 | #warning "Using deprec

In [5]:
from Utils.notebookFunctions import *
import numpy as np
import pandas as pd
import scipy.sparse as sps
import time
import shutil
import optuna
import json
import os
from Utils.seconds_to_biggest_unit import seconds_to_biggest_unit


K_PATH = '/kaggle/working/RECsys_Challenge2024'
GH_PATH = 'TrainedModels/WithKFCV/KNN'

np.random.seed(42)

## Import the repository **RECsys_Challenge2024**

In [6]:
repo = get_repo_from_github(token)

Repository 'RECsys_Challenge2024' found.


In [7]:
config = {
    'model': 'ItemKNNCF',
    'n_folds': 5,
    'metric': 'Recall',
    'tune_parameters': True,
    'database_path': '/kaggle/working/history_ItemKNNCF_Recall.db',
    'copy_prev_best_params': False,
    'tune_best_params': True,
    'save_github': True
}

Import the database where previous tuning trials have been saved.

In [8]:
try:
    shutil.copyfile(
        f'{K_PATH}/{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/history_{config["model"]}_{config["metric"]}.db', 
        config['database_path']
    )
except FileNotFoundError:
    pass # if not present optuna will create it

# Construction of URM and ICM matrices

In [9]:
URM_all_dataframe = pd.read_csv("/kaggle/input/recommender-system-2024-challenge-polimi/data_train.csv")

n_users = len(URM_all_dataframe["user_id"].unique())
n_items = len(URM_all_dataframe["item_id"].unique())

URM_all = sps.csr_matrix((URM_all_dataframe["data"].values, 
                          (URM_all_dataframe["user_id"].values, URM_all_dataframe["item_id"].values)),
                        shape = (n_users, n_items))

URM_all

<35736x38121 sparse matrix of type '<class 'numpy.float64'>'
	with 1764607 stored elements in Compressed Sparse Row format>

In [10]:
ICM_dataframe = pd.read_csv("/kaggle/input/recommender-system-2024-challenge-polimi/data_ICM_metadata.csv")

n_items = len(ICM_dataframe["item_id"].unique())
n_features = len(ICM_dataframe["feature_id"].unique())

ICM_all = sps.csr_matrix((ICM_dataframe["data"].values, 
                          (ICM_dataframe["item_id"].values, ICM_dataframe["feature_id"].values)),
                        shape = (n_items, n_features))

ICM_all

<38121x94331 sparse matrix of type '<class 'numpy.float64'>'
	with 2940040 stored elements in Compressed Sparse Row format>

# Training

In [11]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Data_manager.split_functions.split_train_k_folds import split_train_k_folds

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

folds = split_train_k_folds(URM_all, k=config['n_folds'])

In [12]:
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender

def objective_function_ItemKNNCF(optuna_trial):
    
    recommender_instance = ItemKNNCFRecommender(URM_train)
    similarity = optuna_trial.suggest_categorical("similarity", ['cosine', 'dice', 'jaccard', 'asymmetric', 'tversky', 'euclidean'])
    
    full_hyperp = {"similarity": similarity,
                   "topK": optuna_trial.suggest_int("topK", 0, 750),
                   "shrink": optuna_trial.suggest_int("shrink", 0, 1000),
                   'feature_weighting': optuna_trial.suggest_categorical('feature_weighting', ["BM25", "TF-IDF", "none"])
                  }
    
    if similarity == "asymmetric":
        full_hyperp["asymmetric_alpha"] = optuna_trial.suggest_float("asymmetric_alpha", 0, 2, log=False)
        full_hyperp["normalize"] = True     

    elif similarity == "tversky":
        full_hyperp["tversky_alpha"] = optuna_trial.suggest_float("tversky_alpha", 0, 2, log=False)
        full_hyperp["tversky_beta"] = optuna_trial.suggest_float("tversky_beta", 0, 2, log=False)
        full_hyperp["normalize"] = True 

    elif similarity == "euclidean":
        full_hyperp["normalize_avg_row"] = optuna_trial.suggest_categorical("normalize_avg_row", [True, False])
        full_hyperp["similarity_from_distance_mode"] = optuna_trial.suggest_categorical("similarity_from_distance_mode", ["lin", "log", "exp"])
        full_hyperp["normalize"] = optuna_trial.suggest_categorical("normalize", [True, False])
        
    
    validation_results = []
    
    for fold_idx, (URM_train_fold, URM_validation_fold) in enumerate(folds):
    
        recommender_instance = ItemKNNCFRecommender(URM_train_fold)
        recommender_instance.fit(**full_hyperp)
        
        evaluator = EvaluatorHoldout(URM_validation_fold, cutoff_list=[50])
        result_df, _ = evaluator.evaluateRecommender(recommender_instance)
        
        validation_results.append(result_df.loc[50]["RECALL"])
    
    return np.mean(validation_results)

In [13]:
if config['tune_parameters']:
    
    optuna_study = optuna.create_study(direction='maximize', study_name=f'hyperparameters_tuning_{config["model"]}_{config["metric"]}',
                                storage=f'sqlite:///{config["database_path"]}', load_if_exists=True)

    optuna_study.optimize(objective_function_ItemKNNCF, n_trials=100)

[I 2025-01-04 22:32:42,515] Using an existing study with name 'hyperparameters_tuning_ItemKNNCF_Recall' instead of creating a new one.


Similarity column 38121 (100.0%), 2952.63 column/sec. Elapsed time 12.91 sec
EvaluatorHoldout: Ignoring 173 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35563 (100.0%) in 33.97 sec. Users per second: 1047
Similarity column 38121 (100.0%), 2957.19 column/sec. Elapsed time 12.89 sec
EvaluatorHoldout: Ignoring 163 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35573 (100.0%) in 33.92 sec. Users per second: 1049
Similarity column 38121 (100.0%), 2970.94 column/sec. Elapsed time 12.83 sec
EvaluatorHoldout: Ignoring 150 ( 0.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35586 (100.0%) in 33.99 sec. Users per second: 1047
Similarity column 38121 (100.0%), 2972.89 column/sec. Elapsed time 12.82 sec
EvaluatorHoldout: Ignoring 167 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35569 (100.0%) in 34.02 sec. Users per second: 1045
Similarity column 38121 (100.0%)

[I 2025-01-04 22:36:38,532] Trial 100 finished with value: 0.25917266297874675 and parameters: {'similarity': 'cosine', 'topK': 12, 'shrink': 699, 'feature_weighting': 'TF-IDF'}. Best is trial 99 with value: 0.25964283046146747.


Similarity column 38121 (100.0%), 2939.49 column/sec. Elapsed time 12.97 sec
EvaluatorHoldout: Ignoring 173 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35563 (100.0%) in 37.12 sec. Users per second: 958
Similarity column 38121 (100.0%), 2929.56 column/sec. Elapsed time 13.01 sec
EvaluatorHoldout: Ignoring 163 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35573 (100.0%) in 37.13 sec. Users per second: 958
Similarity column 38121 (100.0%), 2852.57 column/sec. Elapsed time 13.36 sec
EvaluatorHoldout: Ignoring 150 ( 0.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35586 (100.0%) in 37.10 sec. Users per second: 959
Similarity column 38121 (100.0%), 2947.92 column/sec. Elapsed time 12.93 sec
EvaluatorHoldout: Ignoring 167 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35569 (100.0%) in 37.07 sec. Users per second: 959
Similarity column 38121 (100.0%), 29

[I 2025-01-04 22:40:50,995] Trial 101 finished with value: 0.258279823411374 and parameters: {'similarity': 'cosine', 'topK': 50, 'shrink': 677, 'feature_weighting': 'TF-IDF'}. Best is trial 99 with value: 0.25964283046146747.


...

[I 2025-01-05 05:26:35,394] Trial 197 finished with value: 0.260148938520933 and parameters: {'similarity': 'cosine', 'topK': 18, 'shrink': 361, 'feature_weighting': 'TF-IDF'}. Best is trial 172 with value: 0.26030777685753526.


Similarity column 38121 (100.0%), 2866.26 column/sec. Elapsed time 13.30 sec
EvaluatorHoldout: Ignoring 173 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35563 (100.0%) in 46.61 sec. Users per second: 763
Similarity column 38121 (100.0%), 2861.31 column/sec. Elapsed time 13.32 sec
EvaluatorHoldout: Ignoring 163 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35573 (100.0%) in 46.82 sec. Users per second: 760
Similarity column 38121 (100.0%), 2861.38 column/sec. Elapsed time 13.32 sec
EvaluatorHoldout: Ignoring 150 ( 0.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35586 (100.0%) in 46.71 sec. Users per second: 762
Similarity column 38121 (100.0%), 2863.09 column/sec. Elapsed time 13.31 sec
EvaluatorHoldout: Ignoring 167 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35569 (100.0%) in 46.88 sec. Users per second: 759
Similarity column 38121 (100.0%), 28

[I 2025-01-05 05:31:39,270] Trial 198 finished with value: 0.251475362966174 and parameters: {'similarity': 'cosine', 'topK': 332, 'shrink': 368, 'feature_weighting': 'TF-IDF'}. Best is trial 172 with value: 0.26030777685753526.


Similarity column 38121 (100.0%), 2947.90 column/sec. Elapsed time 12.93 sec
EvaluatorHoldout: Ignoring 173 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35563 (100.0%) in 36.72 sec. Users per second: 969
Similarity column 38121 (100.0%), 2954.60 column/sec. Elapsed time 12.90 sec
EvaluatorHoldout: Ignoring 163 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35573 (100.0%) in 36.82 sec. Users per second: 966
Similarity column 38121 (100.0%), 2933.96 column/sec. Elapsed time 12.99 sec
EvaluatorHoldout: Ignoring 150 ( 0.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35586 (100.0%) in 36.57 sec. Users per second: 973
Similarity column 38121 (100.0%), 2914.48 column/sec. Elapsed time 13.08 sec
EvaluatorHoldout: Ignoring 167 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35569 (100.0%) in 36.60 sec. Users per second: 972
Similarity column 38121 (100.0%), 29

[I 2025-01-05 05:35:49,060] Trial 199 finished with value: 0.2592997343508444 and parameters: {'similarity': 'cosine', 'topK': 42, 'shrink': 304, 'feature_weighting': 'TF-IDF'}. Best is trial 172 with value: 0.26030777685753526.


## Some optuna visualizations on recommender parameters

In [14]:
if not config['tune_parameters']:
    optuna_study = optuna.load_study(study_name=f'hyperparameters_tuning_{config["model"]}_{config["metric"]}', storage=f'sqlite:///{config["database_path"]}')
    
fig = optuna.visualization.plot_slice(optuna_study)
fig.show()

In [15]:
if not config['tune_parameters']:
    optuna_study = optuna.load_study(study_name=f'hyperparameters_tuning_{config["model"]}_{config["metric"]}', storage=f'sqlite:///{config["database_path"]}')
    
fig = optuna.visualization.plot_param_importances(optuna_study)
fig.show()

## Let's train the recommender with best parameter values

In [16]:
if config['tune_best_params']:

    if config['tune_parameters']:
        best_params = optuna_study.best_trial.params
    else: 
        with open(f'{K_PATH}/{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/best_params_{config["model"]}_{config["metric"]}.json', 'r') as best_params_json:
            best_params = json.load(best_params_json)

    recommender_instance = ItemKNNCFRecommender(URM_train + URM_validation)
    recommender_instance.fit(**best_params)

Similarity column 38121 (100.0%), 2645.68 column/sec. Elapsed time 14.41 sec


# Testing

Create the recommendations for the submission. 

In [17]:
if config['tune_best_params']:

    data_target_users_test = pd.read_csv('/kaggle/input/recommender-system-2024-challenge-polimi/data_target_users_test.csv')
    create_submission(data_target_users_test, recommender_instance, f'/kaggle/working/submission_{config["model"]}_{config["metric"]}.csv')

Submission file saved as /kaggle/working/submission_ItemKNNCF_Recall.csv


# Save Version on GitHub 

Write or import a json file where best hyperparameters are saved. 

In [18]:
if config['tune_parameters']:
    with open(f'/kaggle/working/best_params_{config["model"]}_{config["metric"]}.json', 'w') as params_file:
        json.dump(optuna_study.best_params, params_file)
        
    if config['save_github']:
        upload_file(
            f'/kaggle/working/best_params_{config["model"]}_{config["metric"]}.json', 
            f'{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/best_params_{config["model"]}_{config["metric"]}.json', 
            f'{config["model"]}_{config["metric"]} tuning results (from kaggle notebook)',
            repo
        )
elif config['copy_prev_best_params']:
    shutil.copyfile(
        f'{K_PATH}/{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/'\
        f'best_params_{config["model"]}_{config["metric"]}.json', 
        f'/kaggle/working/best_params_{config["model"]}_{config["metric"]}.json'
    )

File 'TrainedModels/WithKFCV/KNN/ItemKNNCFRecommender/OptimizingRecall/best_params_ItemKNNCF_Recall.json' updated successfully.


Save the history of the tuned model.

In [19]:
if config['save_github'] and config['tune_parameters']:
    upload_file(
        config['database_path'], 
        f'{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/history_{config["model"]}_{config["metric"]}.db',
        f'Tuning {config["model"]}_{config["metric"]} db updated results (from kaggle notebook)',
        repo
    )

File 'TrainedModels/WithKFCV/KNN/ItemKNNCFRecommender/OptimizingRecall/history_ItemKNNCF_Recall.db' updated successfully.


Save the best trained model and its submission.

In [20]:
if config['save_github'] and config['tune_best_params']: 
    upload_file(
                f'/kaggle/working/submission_{config["model"]}_{config["metric"]}.csv', 
                f'{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/Submission/submission_{config["model"]}_{config["metric"]}.csv', 
                f'New {config["model"]}_{config["metric"]} submission (from kaggle notebook)',
                repo
            )

File 'TrainedModels/WithKFCV/KNN/ItemKNNCFRecommender/OptimizingRecall/Submission/submission_ItemKNNCF_Recall.csv' updated successfully.
